In [73]:
import torch
from torch import nn
from torch.utils.data import DataLoader
#from tqdm.notebook import tqdm
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import datasets
from torchvision.transforms import ToTensor

In [71]:
X = pd.read_csv("data/train_right_time.csv")
X = X.drop(['Time'],axis=1)

#train_tensor = torch.tensor(train.values)
Y = pd.read_csv('data/check_right_time.csv')
Y = Y.drop(['Unnamed: 0'],axis=1)
Y = Y.drop(['ts'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.15)

train = pd.DataFrame(columns=['X','Y'])
test = pd.DataFrame(columns=['X','Y'])

for X, Y in tqdm(zip(X_train.iterrows(), y_train.iterrows()), total=X_train.shape[0]):
    train.loc[len(train.index)]=[X,Y]

for X, Y in tqdm(zip(X_test.iterrows(), y_test.iterrows()), total=X_train.shape[0]):
    train.loc[len(test.index)]=[X,Y]

test = test.to_numpy()
train = train.to_numpy()


#test_tensor = torch.tensor(test.values)

  0%|          | 0/10370 [00:00<?, ?it/s]C:\Users\samue\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\samue\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)
 18%|█▊        | 1831/10370 [00:00<00:03, 2335.58it/s]


In [74]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=None,
)

In [75]:
type(test_data)

torchvision.datasets.mnist.FashionMNIST

In [72]:
test = torch.tensor(test)
train = torch.tensor(train)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
batch_size = 20

# Create data loaders.
train = DataLoader(test, batch_size=batch_size, shuffle=True)
test = DataLoader(train, batch_size=batch_size, shuffle=True)


for X, y in X_test:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

print(len(X_test))
print(len(X_train))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(34, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 6)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def trainf(dataloader, model, loss_fn, optimizer):
    size = len(x_loader.dataset)
    model.double()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def testf(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainf(train, model, loss_fn, optimizer)
    testf(test, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))